#### Alternative ffv implementation.
Very slightly tested.
Will generally perform worse (execution time) than older implementations.
However; can be adjusted to avoid memory limitations.
When performing np.fft.fft, memory equal to n_rows x fft_n x sizeof(float) is required to be allocated. 
This function allows for a workaround by recursively performing np.fft.fft on subsections of the input and returning concatenated output.

In [30]:
import numpy as np

# Requires pre-reshaping of input xs
# Each row should be one epoch, equal to epoch_size in length
def alt_ffv(
    xs,
    Fs = 16000.,
    epoch_size = 2 ** 11,
    fft_n = 2 ** 14,
    lo_f = 12,
    hi_f = 30,
    memory_var = 2 ** 14 # Decrease to allow usage with larger input
):
    frqs = np.fft.fftfreq(fft_n, 1. / Fs)
    lo = np.where(frqs > lo_f)[0][0]
    hi = np.where(frqs > hi_f)[0][0]
    
    # Helper recursive function
    def fftrec(xxs):
        if xxs.shape[0] > memory_var:
            cutoff = int(xxs.shape[0] / 2)
            return np.concatenate((
                fftrec(xxs[0 : cutoff]),
                fftrec(xxs[cutoff :  ])
            ), axis = 0)
        else:
            return np.fft.fft(xxs, fft_n)
        
    fftxs = np.abs( fftrec(xs)[ : , lo : hi] )
    return fftxs, frqs[lo : hi]

#### Stands to benefit from further testing. Should be functionally equivalent to previous ffv, but has not been proven as such.